In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from utils import *
slim = tf.contrib.slim
from tqdm import tqdm_notebook as tqdm
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from mnist_model import generator,discriminator
import utils
l = tf.layers

## Hyperparam

In [2]:
params = dict(
    batch_size = 100,
    seed = 1234,
    z_dim = 2,
    x_dim = 784,
    reg_w = 0.,
    lr = 2e-4,
    logdir = 'gan_mnist/',
    name_model = 'model_latent24'
)
rng = np.random.RandomState(params['seed'])  # seed labels

## Data creation (unl)

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
trainx = np.vstack([mnist.train.images,mnist.validation.images])
trainy = np.hstack([mnist.train.labels,mnist.validation.labels])
testx = mnist.test.images
testy = mnist.test.labels
trainx = np.reshape(trainx,[-1,28,28,1])
testx = np.reshape(testx,[-1,28,28,1])
trainx_unl = trainx.copy()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Build model

In [4]:
tf.reset_default_graph()

In [5]:
unl_dataset = tf.data.Dataset.from_tensor_slices(trainx_unl)
unl_dataset = unl_dataset.shuffle(10000).repeat().batch(params['batch_size'])

iterator_unl = unl_dataset.make_one_shot_iterator()
next_unl = iterator_unl.get_next()

In [6]:
# def generator(z):
#     with tf.variable_scope('generator',reuse=tf.AUTO_REUSE,regularizer=tf.contrib.layers.l2_regularizer(0.001)):
#         h = l.dense(z,356,activation=tf.nn.relu)
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dense(h,784,activation=tf.nn.sigmoid)
#         return h

# def discriminator(x,is_training):
#     with tf.variable_scope('discriminator',reuse=tf.AUTO_REUSE,regularizer=tf.contrib.layers.l2_regularizer(0.001)):
#         h = tf.reshape(x,[-1,784])
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dropout(h,is_training)
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dropout(h,is_training)
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dropout(h,is_training)
#         h = l.dense(h,356,activation=tf.nn.relu)
#         h = l.dropout(h,is_training)
#         h = l.dense(h,1,activation=tf.nn.sigmoid)
#         return h

In [7]:
is_training_pl = tf.placeholder(tf.bool, [], name='is_training_pl')
data = tf.cast(next_unl,tf.float32)
noise = tf.random_normal(shape=[params['batch_size'],params['z_dim']])
# Construct generator and discriminator net
# samples = generator(noise)
# real_score = discriminator(data,is_training_pl)
# fake_score = discriminator(samples,is_training_pl)
samples = generator(noise,is_training=is_training_pl,reuse=tf.AUTO_REUSE)
real_score = discriminator(data,is_training=is_training_pl,reuse=tf.AUTO_REUSE)
fake_score = discriminator(samples,is_training=is_training_pl,reuse=tf.AUTO_REUSE)

gen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "generator")
disc_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "discriminator") 

loss_g = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_score,labels=tf.ones_like(fake_score)))
loss_d = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real_score,labels=tf.ones_like(fake_score)))+\
tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_score,labels=tf.zeros_like(fake_score)))

loss_g += tf.reduce_sum(tf.losses.get_regularization_losses())
loss_d += tf.reduce_sum(tf.losses.get_regularization_losses())

Tensor("discriminator/layer_3/leaky_relu/sub:0", shape=(?, 4, 4, 512), dtype=float32)
Tensor("discriminator_1/layer_3/leaky_relu/sub:0", shape=(100, 4, 4, 512), dtype=float32)


In [8]:
disc_vars

[<tf.Variable 'discriminator/layer_1/conv/kernel:0' shape=(4, 4, 1, 128) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_1/conv/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_2/conv/kernel:0' shape=(4, 4, 128, 256) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_2/conv/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_2/batch_normalization/gamma:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_2/batch_normalization/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_3/conv/kernel:0' shape=(4, 4, 256, 512) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_3/conv/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_3/batch_normalization/gamma:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'discriminator/layer_3/batch_normalization/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'discriminator/dense/kernel:0' shape=(512, 1) dtype=float32_

In [8]:
optimizer_dis = tf.train.AdamOptimizer(learning_rate=params['lr'],
                                               beta1=0.5, name='dis_optimizer')
optimizer_gen = tf.train.AdamOptimizer(learning_rate=params['lr'],
                                               beta1=0.5, name='gen_optimizer')

update_ops_gen = tf.get_collection(tf.GraphKeys.UPDATE_OPS,
                                   scope='generator')
update_ops_dis = tf.get_collection(tf.GraphKeys.UPDATE_OPS,
                                   scope='discriminator')

with tf.control_dependencies(update_ops_gen): # attached op for moving average batch norm
    traing = optimizer_gen.minimize(loss_g,var_list=gen_vars)
with tf.control_dependencies(update_ops_dis):
    traind = optimizer_dis.minimize(loss_d,var_list=disc_vars)

In [13]:
print('rms-georgios')
optimizer = tf.train.RMSPropOptimizer(0.001, use_locking=True)

d_grads = tf.gradients(loss_d, disc_vars)
g_grads = tf.gradients(loss_g, gen_vars)

variables = disc_vars + gen_vars
grads = d_grads + g_grads

apply_vec = [(g + 0.001 * v, v)for (g, v) in zip(grads, variables)]

update_ops_gen = tf.get_collection(tf.GraphKeys.UPDATE_OPS,
                                   scope='generator')
with tf.control_dependencies(update_ops_gen): # attached op for moving average batch norm
    train_op = optimizer.apply_gradients(apply_vec)

rms-georgios


In [15]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [17]:
fs = []
frames = []
np_samples = []
n_batches_viz = 1
viz_every = 1000
lsd=[]
lsg=[]
for i in tqdm(range(20000)):
    ld,lg, _= sess.run([loss_d,loss_g, train_op],{is_training_pl:True})
#     ld,_=sess.run([loss_d,traind],{is_training_pl:True})
#     lg,_=sess.run([loss_g, traing],{is_training_pl:True})
    lsd.append(ld); lsg.append(lg);

    if (i) % viz_every == 0:
        xx, yy = sess.run([data,samples],{is_training_pl:False})
#         print('iter nr : ',gstep)
        plt.figure(figsize=(10,5))
#         plt.subplot(121)
#         show_digits(xx[:100])
#         plt.subplot(122)
        show_digits(yy[:100])
        plt.show()


HBox(children=(IntProgress(value=0, max=20000), HTML(value='')))

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/bruno/anaconda3/envs/tensorflow/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/bruno/.local/lib/python3.5/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/home/bruno/anaconda3/envs/tensorflow/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



ResourceExhaustedError: OOM when allocating tensor with shape[512,256,4,4]
	 [[Node: generator/layer_4/conv/conv2d_transpose = Conv2DBackpropInput[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](generator/layer_4/conv/stack, generator/layer_4/conv/kernel/read, generator/layer_2/relu)]]
	 [[Node: add_2/_85 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2585_add_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'generator/layer_4/conv/conv2d_transpose', defined at:
  File "/home/bruno/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/bruno/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exe

In [ ]:
if not os.path.exists(params['logdir']):
    os.makedirs(params['logdir'])
saver.save(sess, os.path.join('./saved_models/',params['logdir'],params['name_model']))